# Feature Engineering
Explanation:

## Import Packages

In [1]:
from importnb import Notebook
import pandas as pd
import numpy as np

## Dataframe import from 'DataPreprocessing'

In [2]:
with Notebook():
    from DataPreprocessing_Pipeline import companies, funding, investors

Index(['ID', 'Organization Name', 'Industries', 'Description',
       'CB Rank (Company)', 'Founded Date', 'Exit Date', 'Website', 'Twitter',
       'Facebook', 'LinkedIn', 'Full Description', 'Industry Groups',
       'Number of Founders', 'Number of Employees', 'Founders',
       'Number of Funding Rounds', 'Last Funding Date', 'Last Funding Amount',
       'Last Funding Type', 'Total Funding Amount',
       'Total Funding Amount (in USD)', 'Top 5 Investors',
       'Number of Lead Investors', 'Number of Investors',
       'Number of Acquisitions', 'Acquisition Status', 'Acquired by',
       'Announced Date Acquisition', 'IPO'],
      dtype='object')
Index(['Funding ID', 'Organization Name', 'Funding Type', 'Money Raised',
       'Announced Date', 'Lead Investors', 'Number of Investors',
       'Investor Names', 'Company ID'],
      dtype='object')
Index(['Investor ID', 'Organization/Person Name', 'Investor Type',
       'Number of Investments', 'Number of Exits', 'Location', 'Descri

## New Features: Companies <> Investment Rounds
Explanation:

### Feature 1: Months Until First Round
Number of months between the funding date and the first investment round. Explanation: The table 'all_rounds' has every investment round made during the existence of the companies. We already mapped the respectively funding round to the startup. 

In [3]:
def calculate_months_until_first_round(company_id, founded_date):
    rounds = funding[funding['Company ID'] == company_id]

    if rounds.empty:
        return np.nan

    first_round_date = rounds['Announced Date'].min()

    delta_months = (first_round_date.year - founded_date.year) * 12 + (first_round_date.month - founded_date.month)
    return max(delta_months, 0)

companies['Months until First Round'] = companies.apply(
    lambda row: calculate_months_until_first_round(row['ID'], row['Founded Date']),
    axis=1
)

companies['Months until First Round'] = companies['Months until First Round'].fillna(-1)

### Feature 2: Grant Y/N
If the company received a Grant, they get a Yes (1). If not, a No (0). For VCs it is desirable to have a company with a Grant because it leverages the invested money as a Grant do not dilute existing shareholders.

In [4]:
def check_grant_for_company(company_id):
    company_rounds = funding[funding['Company ID'] == company_id]
    has_grant = any(company_rounds['Funding Type'] == 'Grant')
    return 1 if has_grant else 0

companies['Grant Y/N'] = companies['ID'].apply(check_grant_for_company)

### Feature 3: Last Round Type
Explanation:

In [5]:
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])

last_round = (
    funding.sort_values(by=['Company ID', 'Announced Date'])
    .groupby('Company ID')
    .last()
    .reset_index()
)

last_round = last_round[['Company ID', 'Funding Type']]

companies = companies.merge(
    last_round,
    left_on='ID',
    right_on='Company ID',
    how='left'
)

companies = companies.rename(columns={'Funding Type': 'Last Round Type'})

companies.drop(columns=['Company ID'], inplace=True)

companies['Last Round Type'] = companies['Last Round Type'].fillna('No Funding')

companies.drop(columns=['Last Funding Type'], inplace=True)

### Feature 4: Acquisition Status 'Was Acquired', 'Made Acquisitions'
Explanation:

In [6]:
if 'Acquisition Status' in companies.columns:
    unique_values = companies['Acquisition Status'].dropna().unique()
    print(unique_values)

['Was Acquired' 'Made Acquisitions' 'Made Acquisitions, Was Acquired']


In [7]:
companies['Was Acquired'] = 0
companies['Made Acquisitions'] = 0

companies.loc[companies['Acquisition Status'] == 'Was Acquired', 'Was Acquired'] = 1
companies.loc[companies['Acquisition Status'] == 'Made Acquisitions', 'Made Acquisitions'] = 1
companies.loc[companies['Acquisition Status'] == 'Made Acquisitions, Was Acquired', ['Made Acquisitions', 'Was Acquired']] = 1

companies.drop(columns=['Acquisition Status'], inplace=True)

### Feature 5: Funding phases
Explanation:

In [8]:
if 'Last Round Type' in companies.columns:
    distinct_funding_types = companies['Last Round Type'].dropna().unique()
    print(distinct_funding_types)
else:
    print("The column 'Funding Type' does not exist in the DataFrame.")

['Seed' 'Pre-Seed' 'Venture - Series Unknown' 'Series A' 'Series C'
 'No Funding' 'Grant' 'Convertible Note' 'Private Equity' 'Series B'
 'Angel' 'Series D' 'Post-IPO Debt' 'Post-IPO Secondary' 'Series E'
 'Post-IPO Equity' 'Series F']


In [9]:
funding_columns = ['Project Funding', 'Startup Funding', 'Growth Funding', 'Expansion Funding', 'Exit Funding']
for col in funding_columns:
    companies[col] = 0

funding_mapping = {
    'Project Funding': ['Angel', 'Pre-Seed', 'Convertible Note', 'Grant', 'Venture - Series Unknown'],
    'Startup Funding': ['Seed', 'Venture - Series Unknown'],
    'Growth Funding': ['Series A', 'Series B', 'Series C'],
    'Expansion Funding': ['Series D', 'Series E', 'Series F'],
    'Exit Funding': ['Private Equity', 'Post-IPO Secondary', 'Post-IPO Equity']
}

for funding_type, types in funding_mapping.items():
    companies.loc[companies['Last Round Type'].isin(types), funding_type] = 1

companies.loc[companies['Exit Funding'] == 1, funding_columns[:-1]] = 1
companies.loc[companies['Expansion Funding'] == 1, ['Growth Funding', 'Startup Funding', 'Project Funding']] = 1
companies.loc[companies['Growth Funding'] == 1, ['Startup Funding', 'Project Funding']] = 1
companies.loc[companies['Startup Funding'] == 1, 'Project Funding'] = 1

### HighFunding (Series B or higher)

In [10]:
high_funding_rounds = ['Series B', 'Series C', 'Series D', 'Series E', 'Series F', 'Private Equity', 'Post-IPO Equity']
companies['HighFunding'] = companies['Last Round Type'].isin(high_funding_rounds).astype(int)

In [11]:
companies

,ID,Organization Name,Industries,Description,CB Rank (Company),Founded Date,Exit Date,Website,Twitter,Facebook,...,Grant Y/N,Last Round Type,Was Acquired,Made Acquisitions,Project Funding,Startup Funding,Growth Funding,Expansion Funding,Exit Funding,HighFunding
0,000001,Axino Solutions,"[Computer, Software]",Axino Solutions is an innovative software and ...,"2,266,795",2016-01-01,2021-04-06,http://www.axino-group.com,https://twitter.com/axinosolutions,https://www.facebook.com/Axino-Solutions-29835...,...,0,Seed,1,0,1,1,0,0,0,0
1,000002,cellEQ,[Professional Services],"CellEQ develops, produces and distributes indi...","200,169",2018-01-01,NaT,https://celleq.de,NaN,NaN,...,0,Pre-Seed,0,0,1,0,0,0,0,0
2,000003,Clinomic,"[Artificial Intelligence (AI), Biotechnology, ...",Medical AI company with a focus on critical care.,"27,155",2019-01-14,NaT,https://clinomic.ai,https://twitter.com/clinomicai,https://www.facebook.com/ClinomicAI/,...,0,Venture - Series Unknown,0,0,1,1,0,0,0,0
3,000004,crop.zone,"[Agriculture, AgTech, Farming]",crop.zone provides farmers with chemistry-free...,"30,069",2019-12-01,NaT,https://crop.zone/,https://twitter.com/crop_zone,https://www.facebook.com/crop.zone/,...,0,Series A,0,0,1,1,1,0,0,0
4,000005,Dropslab,"[Apps, Information Technology, Software]",Dropslab provides a SaaS platform for AR-based...,"121,887",2018-01-01,NaT,https://dropslab.com,NaN,https://www.facebook.com/DropslabTechnologies,...,0,Seed,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1505,001506,Veprosa,"[E-Commerce, Food and Beverage, Food Processing]",Veprosa comprises a protein complex of rice an...,"155,607",2019-01-01,NaT,https://veprosa.com,NaN,https://www.facebook.com/veganproteinsauce,...,0,Angel,0,0,1,0,0,0,0,0
1506,001507,vialytics,"[Big Data, Government, GovTech, SaaS, Software]",vialytics' mission is to digitize road managem...,"7,584",2017-01-01,NaT,http://www.vialytics.de,https://twitter.com/vialytics,https://www.facebook.com/Vialytics-25545635179...,...,0,Venture - Series Unknown,0,0,1,1,0,0,0,0
1507,001508,ViveLaCar,"[Automotive, E-Commerce, E-Commerce Platforms,...",ViveLaCar offers car subscription delivered th...,"226,205",2019-01-16,2023-01-24,https://www.vivelacar.com/,NaN,https://www.facebook.com/vivelacar/,...,0,Seed,1,1,1,1,0,0,0,0
1508,001509,Waays CrowdCoaching,"[Software, Sports, Training]",Waays CrowdCoaching offers a crowd-coaching ma...,"2,778,981",2017-01-01,NaT,http://www.waays.de,NaN,NaN,...,0,No Funding,0,0,0,0,0,0,0,0


In [12]:
funding

,Funding ID,Organization Name,Funding Type,Money Raised,Announced Date,Lead Investors,Number of Investors,Investor Names,Company ID
0,000001,( OFFTIME ),Seed,NaN,2015-01-01,Axel Springer Plug and Play Accelerator,1.0,[Axel Springer Plug and Play Accelerator],000000
1,000002,12K Research,Convertible Note,NaN,2015-10-02,NaN,1.0,[Project Flying Elephant],000033
2,000003,1aim,Angel,NaN,2015-03-01,NaN,2.0,"[Brent Hoberman, Matthias Ummenhofer]",000000
3,000004,3S Antriebe GmbH,Grant,50000.0,2015-07-01,EASME,1.0,[EASME],000000
4,000005,3YOURMIND,Seed,1500000.0,2015-12-15,AM Ventures,1.0,[AM Ventures],000000
...,...,...,...,...,...,...,...,...,...
6863,006864,vialytics,Venture - Series Unknown,NaN,2019-07-30,EnBW New Ventures,1.0,[EnBW New Ventures],001507
6864,006865,VISPA,Seed,1500000.0,2022-11-17,NaN,3.0,"[ES KAPITAL, MBG Baden Wuerttemberg, Volksbank...",000000
6865,006866,ViveLaCar,Pre-Seed,NaN,2019-05-20,Vector Venture Capital,1.0,[Vector Venture Capital],001508
6866,006867,ViveLaCar,Seed,NaN,2020-01-28,Die Vienna Insurance Group,2.0,"[Die Vienna Insurance Group, Vector Venture Ca...",001508


### Feature 6: Average Time To Next Round
Explanation:

In [13]:
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])
companies['Founded Date'] = pd.to_datetime(companies['Founded Date'])

def calculate_avg_time_to_next_round(company_id, company_founded_date, funding_df):
    company_funding = funding_df[funding_df['Company ID'] == company_id]

    if len(company_funding) == 1:
        time_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        avg_time_to_next_round = time_diff / 30
    elif len(company_funding) >= 2:
        time_diffs = []
        first_round_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        time_diffs.append(first_round_diff)
        company_funding = company_funding.sort_values('Announced Date')
        for i in range(1, len(company_funding)):
            time_diff = (company_funding['Announced Date'].iloc[i] - company_funding['Announced Date'].iloc[i-1]).days
            time_diffs.append(time_diff)
        avg_time_to_next_round = sum(time_diffs) / len(time_diffs) / 30
    else:
        avg_time_to_next_round = None

    if pd.isna(avg_time_to_next_round) or avg_time_to_next_round < -1:
        avg_time_to_next_round = -1

    return avg_time_to_next_round

companies['Average Time To Next Round'] = companies.apply(
    lambda row: calculate_avg_time_to_next_round(row['ID'], row['Founded Date'], funding), axis=1
)

### Feature 7: Average Funding Size
Explanation:

In [14]:
funding_summary = funding.groupby('Company ID').agg(
    TotalMoneyRaised=('Money Raised', 'sum'),
    TotalRounds=('Money Raised', 'count')
).reset_index()

funding_summary['Average Funding Size'] = funding_summary['TotalMoneyRaised'] / funding_summary['TotalRounds']

companies = companies.merge(
    funding_summary[['Company ID', 'Average Funding Size']],
    left_on='ID',
    right_on='Company ID',
    how='left'
)

companies['Average Funding Size'] = companies['Average Funding Size'].fillna(0)

companies.drop(columns=['Company ID'], inplace=True)

## New Features: Companies <> Investors
Explanation:

### Feature 1-4: Average Number of Investments by Investors
Explanation:

In [15]:
funding_exploded = funding.explode('Investor Names')
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()
investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

merged = funding_exploded.merge(
    investors,
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

columns_to_process = ['Number of Investments', 'Number of Exits', 'Number of Lead Investments', 'Number of Portfolio Organizations']
for col in columns_to_process:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')

not_found_investors = merged[merged['Number of Investments'].isnull()]['Investor Names'].unique()
if len(not_found_investors) > 0:
    print(f"{len(not_found_investors)} investors could not be found.")
else:
    print("All investors were successfully found.")

not_found_investors_df = pd.DataFrame(not_found_investors, columns=['Investor Names'])

if 'Company ID' in companies.columns:
    companies = companies.drop(columns=['Company ID'])

for col in columns_to_process:
    col_average = merged.groupby('Company ID')[col].mean().reset_index()
    companies = companies.merge(
        col_average,
        left_on='ID',
        right_on='Company ID',
        how='left',
        suffixes=('', '_drop')
    )
    companies[col] = companies[col].fillna(0)
    companies = companies.rename(columns={col: f'Average {col} by Investors'})

companies = companies.drop(columns=[col for col in companies.columns if col.endswith('_drop')], errors='ignore')


3608 investors could not be found.


### Feature 5: Origin Country of Investors
Explanation:

In [16]:
funding_exploded = funding.explode('Investor Names')
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()
investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

merged = funding_exploded.merge(
    investors[['Organization/Person Name', 'Country']],
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

distinct_countries = investors['Country'].dropna().unique()[:5]

country_encoded = pd.get_dummies(merged['Country'], prefix='Investor Country', dtype=int)

country_aggregated = country_encoded.groupby(merged['Company ID']).max()

companies = companies.merge(
    country_aggregated,
    left_on='ID',
    right_index=True,
    how='left'
)

for country in distinct_countries:
    column_name = f'Investor Country_{country}'
    if column_name in companies.columns:
        companies[column_name] = companies[column_name].fillna(0)

### Feature 6: Top Investor Participation
Identifies whether a company has investors with a high number of exits.

In [17]:
top_investors = investors[investors['Number of Exits'] > investors['Number of Exits'].mean()]['Organization/Person Name']
def has_top_investor(company_id, funding_df, top_investors):
    company_funding = funding_df[funding_df['Company ID'] == company_id]
    if company_funding.empty:
        return 0
    investors_list = company_funding['Investor Names'].explode()
    return 1 if any(investor in top_investors.values for investor in investors_list) else 0

companies['Top Investor Participation'] = companies['ID'].apply(
    lambda x: has_top_investor(x, funding, top_investors)
)

## New Features: Companies

### Feature 1: 'Months between Founding and Acquisition'

In [18]:
companies['Months between Founding and Acquisition'] = companies.apply(
    lambda row: (row['Announced Date Acquisition'] - row['Founded Date']).days // 30
    if pd.notna(row['Announced Date Acquisition']) else -1,
    axis=1
)

### Feature 2: Category One Hot Encoding

In [19]:
exploded = companies.explode('Industry Groups')
exploded['Industry Groups'] = exploded['Industry Groups'].str.strip().str.lower()
exploded = exploded[exploded['Industry Groups'].notna() & (exploded['Industry Groups'] != '')]

one_hot_encoded = pd.get_dummies(exploded['Industry Groups'], prefix='Industry', dtype=int)
one_hot_aggregated = one_hot_encoded.groupby(exploded.index).max()

companies = pd.concat([companies, one_hot_aggregated], axis=1)

In [20]:
pd.set_option('display.max_columns', None)
print(companies)

          ID    Organization Name  \
0     000001      Axino Solutions   
1     000002               cellEQ   
2     000003             Clinomic   
3     000004            crop.zone   
4     000005             Dropslab   
...      ...                  ...   
1505  001506              Veprosa   
1506  001507            vialytics   
1507  001508            ViveLaCar   
1508  001509  Waays CrowdCoaching   
1509  001510             WildFyre   

                                             Industries  \
0                                  [Computer, Software]   
1                               [Professional Services]   
2     [Artificial Intelligence (AI), Biotechnology, ...   
3                        [Agriculture, AgTech, Farming]   
4              [Apps, Information Technology, Software]   
...                                                 ...   
1505   [E-Commerce, Food and Beverage, Food Processing]   
1506    [Big Data, Government, GovTech, SaaS, Software]   
1507  [Automotive, E-Com

In [21]:
companies

,ID,Organization Name,Industries,Description,CB Rank (Company),Founded Date,Exit Date,Website,Twitter,Facebook,LinkedIn,Full Description,Industry Groups,Number of Founders,Number of Employees,Founders,Number of Funding Rounds,Last Funding Date,Last Funding Amount,Total Funding Amount,Total Funding Amount (in USD),Top 5 Investors,Number of Lead Investors,Number of Investors,Number of Acquisitions,Acquired by,Announced Date Acquisition,IPO,Months until First Round,Grant Y/N,Last Round Type,Was Acquired,Made Acquisitions,Project Funding,Startup Funding,Growth Funding,Expansion Funding,Exit Funding,HighFunding,Average Time To Next Round,Average Funding Size,Company ID,Average Number of Investments by Investors,Average Number of Exits by Investors,Average Number of Lead Investments by Investors,Average Number of Portfolio Organizations by Investors,Investor Country_AUS,Investor Country_GER,Investor Country_SWI,Investor Country_UK,Investor Country_USA,Top Investor Participation,Months between Founding and Acquisition,Industry_administrative services,Industry_advertising,Industry_agriculture and farming,Industry_apps,Industry_artificial intelligence (ai),Industry_biotechnology,Industry_blockchain and cryptocurrency,Industry_clothing and apparel,Industry_commerce and shopping,Industry_community and lifestyle,Industry_consumer electronics,Industry_consumer goods,Industry_content and publishing,Industry_data and analytics,Industry_design,Industry_education,Industry_energy,Industry_events,Industry_financial services,Industry_food and beverage,Industry_gaming,Industry_government and military,Industry_hardware,Industry_health care,Industry_information technology,Industry_internet services,Industry_lending and investments,Industry_manufacturing,Industry_media and entertainment,Industry_messaging and telecommunications,Industry_mobile,Industry_music and audio,Industry_natural resources,Industry_navigation and mapping,Industry_other,Industry_payments,Industry_platforms,Industry_privacy and security,Industry_professional services,Industry_real estate,Industry_sales and marketing,Industry_science and engineering,Industry_social impact,Industry_software,Industry_sports,Industry_sustainability,Industry_transportation,Industry_travel and tourism,Industry_video
0,000001,Axino Solutions,"[Computer, Software]",Axino Solutions is an innovative software and ...,"2,266,795",2016-01-01,2021-04-06,http://www.axino-group.com,https://twitter.com/axinosolutions,https://www.facebook.com/Axino-Solutions-29835...,https://www.linkedin.com/company/axino-solutio...,Axino Solutions is an innovative software and ...,"[Consumer Electronics, Hardware, Software]",2.555556,175.5,NaN,2,2022-08-18,NaN,NaN,NaN,"Technology Fund, Momenta",2.0,2.0,0.0,Axians IT Business Solutions GmbH,2021-04-06,0,62.0,0,Seed,1,0,1,1,0,0,0,0,62.933333,0.000000e+00,000001,71.000000,8.000000,35.000000,56.0,0.0,0.0,1.0,0.0,0.0,0,64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,000002,cellEQ,[Professional Services],"CellEQ develops, produces and distributes indi...","200,169",2018-01-01,NaT,https://celleq.de,NaN,NaN,https://www.linkedin.com/company/celleq-gmbh,"CellEQ develops, produces and distributes indi...",[Other],2.000000,5.5,"Florian Kessler, Phillip Nicolas Wagner",1,2018-01-01,30000.0,30000.0,30000.0,EXIST,0.0,1.0,0.0,NaN,NaT,0,0.0,0,Pre-Seed,0,0,1,0,0,0,0,0,0.000000,3.000000e+04,000002,54.000000,2.000000,15.000000,52.0,0.0,1.0,0.0,0.0,0.0,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,000003,Clinomic,"[Artificial Intelligence (AI), Biotechnology, ...",Medical AI company with a focus on critical care.,"27,155",2019-01-14,NaT,https://clinomic.ai,https://twitter.com/clinomicai,https://ww